In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import os 
import random 
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime 
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from surprise import Reader,Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import GridSearchCV


Data Extraction

In [2]:
#function that insert combined data text file into netflix rating csv file 
def load_data():
    #create new folder netflix_csv to store txt data into csv format
    netflix_csv_file = open('netflix_rating.csv',mode='w')
    rating_files = ['combined_data_2.txt']
    for file in rating_files:
        #To open the combined_data.txt file as f
        with open(file) as f:
            for line in f:
                #Remove evey space between each line 
                line = line.strip()
                if line.endswith(":"):
                    movie_id = line.replace(":","")
                else:
                    row_data = []
                    row_data = [item for item in line.split(",")]
                    row_data.insert(0,movie_id)
                    netflix_csv_file.write(",".join(row_data))
                    netflix_csv_file.write('\n')
    netflix_csv_file.close()
    df = pd.read_csv('netflix_rating.csv',sep=',',names=["movie_id","customer_id","rating","date"])
    return df



In [4]:
netflix_rating_df = load_data()
netflix_rating_df.head()




,movie_id,customer_id,rating,date
0,4500,2532865,4,2005-07-26
1,4500,573364,3,2005-06-20
2,4500,1696725,3,2004-02-27
3,4500,1253431,3,2004-03-31
4,4500,1265574,2,2003-09-01
